In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
np.random.seed(2)
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPool2D, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
from PIL import Image, ImageChops, ImageEnhance
import os
import itertools
import random

In [ ]:
def convert_to_ela_image(path, quality):
    temp_filename = 'temp_file_name.jpg'
    ela_filename = 'temp_ela.png'
    
    image = Image.open(path).convert('RGB')
    image.save(temp_filename, 'JPEG', quality = quality)
    temp_image = Image.open(temp_filename)
    
    ela_image = ImageChops.difference(image, temp_image)
    
    extrema = ela_image.getextrema()
    max_diff = max([ex[1] for ex in extrema])
    if max_diff == 0:
        max_diff = 1
    scale = 255.0 / max_diff
    
    ela_image = ImageEnhance.Brightness(ela_image).enhance(scale)
    
    return ela_image

In [ ]:
# real_image_path = 'Au_cha_00027.jpg'
# Image.open(real_image_path)

In [ ]:
# au = convert_to_ela_image(real_image_path, 95)
# au.save('au.png')

In [ ]:
# fake_image_path = 'Tp_D_NNN_M_N_pla10121_cha00027_11669.jpg'
# Image.open(fake_image_path)

In [ ]:
# Tp = convert_to_ela_image(fake_image_path, 95)
# Tp.save('tp.png')

In [ ]:
image_size = (224, 224)
def prepare_image(image_path):
    return np.array(convert_to_ela_image(image_path, 90).resize(image_size)).flatten() / 255.0

In [ ]:
X = [] # ELA converted images
Y = [] # 0 for fake, 1 for real

In [ ]:
def prepare_data(path, cls):
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            if filename.endswith('jpg') or filename.endswith('png'):
                try:
                    full_path = os.path.join(dirname, filename)
                    X.append(prepare_image(full_path))
                    Y.append(cls)
                except:
                    pass
                if len(Y) % 500 == 0:
                    print('Processing {} images'.format(len(Y)))

In [ ]:
au_path = 'D:\Google Drive\CASIA2\Au'
prepare_data(au_path, 1)
random.shuffle(X)
X = X[:2100]
Y = Y[:2100]
print(len(X), len(Y))

In [ ]:
tp_path = 'D:\Google Drive\CASIA2\Tp'
prepare_data(tp_path, 0)
print(len(X), len(Y))

In [ ]:
X = np.array(X)
Y = to_categorical(Y, 2)
X = X.reshape(-1, 224, 224, 3)

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size = 0.2, random_state=5)

In [ ]:
X = X.reshape(-1,1,1,1)
print(len(X_train), len(Y_train))
print(len(X_val), len(Y_val))

In [ ]:
X.shape

In [ ]:
def build_model():
    model = Sequential()
    model.add(Conv2D(32, 5, padding='valid', activation='relu', input_shape=[224, 224, 3]))
    model.add(Conv2D(32, 5, padding='valid', activation='relu', input_shape=[224, 224, 3]))
    model.add(MaxPool2D(pool_size=2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(2, activation='softmax'))
    return model

In [ ]:
model = build_model()
model.summary()

In [ ]:
epochs = 30
batch_size = 32
init_lr = 1e-4
optimizer = Adam(lr = init_lr, decay = init_lr/epochs)

In [ ]:
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_acc', min_delta=0,patience=2, verbose=0, mode='auto')

checkpoint_filepath = 'ela/checkpoint'
model_checkpoint_callback = ModelCheckpoint(filepath=checkpoint_filepath, save_weights_only=True, monitor='val_accuracy', mode='max', save_best_only=True)

In [ ]:
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_val, Y_val), callbacks=[early_stopping, model_checkpoint_callback])

In [ ]:
model.save('model_casia_run1.h5')

In [ ]:
from sklearn.metrics import average_precision_score

predictions = model.predict(X_val)
average_precision = average_precision_score(Y_val, predictions)

In [ ]:
# from sklearn.metrics import precision_recall_curve
# from sklearn.metrics import plot_precision_recall_curve

# disp = plot_precision_recall_curve(model, X_val, Y_val)
# disp.ax_.set_title('2-class Precision-Recall curve: '
#                    'AP={0:0.2f}'.format(average_precision))

In [ ]:
plt.plot(history.history['loss'], color='b', label="Training loss")
plt.plot(history.history['val_loss'], color='r', label="validation loss")
legend = plt.legend(loc='best', shadow=True)
plt.savefig("loss.png", bbox_inches='tight', pad_inches=0)

In [ ]:
plt.plot(history.history['accuracy'], color='b', label="Training accuracy")
plt.plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = plt.legend(loc='best', shadow=True)
plt.savefig("acc.png", bbox_inches='tight', pad_inches=0)

In [ ]:
# Plot the loss and accuracy curves for training and validation 
fig, ax = plt.subplots(2,1)
ax[0].plot(history.history['loss'], color='b', label="Training loss")
ax[0].plot(history.history['val_loss'], color='r', label="validation loss",axes =ax[0])
legend = ax[0].legend(loc='best', shadow=True)

ax[1].plot(history.history['accuracy'], color='b', label="Training accuracy")
ax[1].plot(history.history['val_accuracy'], color='r',label="Validation accuracy")
legend = ax[1].legend(loc='best', shadow=True)

In [ ]:
model = tf.keras.models.load_model('model_casia_run1.h5')

In [ ]:
class_names = ['fake', 'real']

In [ ]:
def predict_class(file_paths):
    for file_path in file_paths:
        image = prepare_image(file_path)
        image = image.reshape(-1, 128, 128, 3)
        y_pred = model.predict(image)
        y_pred_class = np.argmax(y_pred, axis = 1)[0]
        print(f'Class: {class_names[y_pred_class]} Confidence: {np.amax(y_pred) * 100:0.2f}')

In [ ]:
fake_image_paths = ['Tp_1407_278731_138.97_198.88_314.05_553.99_person.png','Tp_71_71880_47.87_185.72_531.3_305.38_train.png']
real_image_paths = ['COCO_train2014_000000000025.jpg', '../CASIA2/Au/Au_ani_00001.jpg', '../CASIA2/Au/Au_ani_00001.jpg']

In [ ]:
predict_class(real_image_paths)

In [ ]:
predict_class(fake_image_paths)

In [ ]:
image = prepare_image('Tp_1407_278731_138.97_198.88_314.05_553.99_person.png')
image = image.reshape(-1, 128, 128, 3)
y_pred = model.predict(image)
y_pred_class = np.argmax(y_pred, axis = 1)[0]
print(f'Class: {y_pred_class} Confidence: {np.amax(y_pred) * 100:0.2f}')

In [ ]:
y_pred.shape